In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain.agents import AgentState

class CustomState(AgentState):
    favourite_colour: str

## Write to state

In [4]:
from langchain.tools import tool, ToolRuntime
from langgraph.types import Command
from langchain.messages import ToolMessage

@tool
def update_favourite_colour(favourite_colour: str, runtime: ToolRuntime) -> Command:
    """Update the favourite colour of the user in the state once they've revealed it."""
    return Command(update={
        "favourite_colour": favourite_colour, 
        "messages": [ToolMessage("Successfully updated favourite colour", tool_call_id=runtime.tool_call_id)]}
        )

In [5]:
from langchain.agents import create_agent
from langgraph.checkpoint.memory import InMemorySaver

from langchain_nvidia_ai_endpoints import ChatNVIDIA

model = ChatNVIDIA(model = "meta/llama-3.3-70b-instruct", temperature=0.2)

agent = create_agent(
    model=model,
    tools=[update_favourite_colour],
    checkpointer=InMemorySaver(),
    state_schema=CustomState
)

In [6]:
from langchain.messages import HumanMessage

response = agent.invoke(
    { "messages": [HumanMessage(content="My favourite colour is green")]},
    {"configurable": {"thread_id": "1"}}
)

In [7]:
from pprint import pprint

pprint(response)

{'favourite_colour': 'green',
 'messages': [HumanMessage(content='My favourite colour is green', additional_kwargs={}, response_metadata={}, id='0e9bf8fc-72c8-4806-9b05-9e3ec062a24f'),
              AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'chatcmpl-tool-b5c864eb7d8d05fd', 'type': 'function', 'function': {'name': 'update_favourite_colour', 'arguments': '{"favourite_colour": "green"}'}}]}, response_metadata={'role': 'assistant', 'content': None, 'refusal': None, 'annotations': None, 'audio': None, 'function_call': None, 'tool_calls': [{'id': 'chatcmpl-tool-b5c864eb7d8d05fd', 'type': 'function', 'function': {'name': 'update_favourite_colour', 'arguments': '{"favourite_colour": "green"}'}}], 'reasoning': None, 'reasoning_content': None, 'token_usage': {'prompt_tokens': 255, 'total_tokens': 282, 'completion_tokens': 27, 'prompt_tokens_details': None}, 'finish_reason': 'tool_calls', 'model_name': 'meta/llama-3.3-70b-instruct'}, id='lc_run--019c8d7a-dfc1-7cd3-8158-33770

In [ ]:
response = agent.invoke(
    { 
        "messages": [HumanMessage(content="Hello, how are you?")],
        "favourite_colour": "green" # pass update of state directly in the invoke call
    },
    {"configurable": {"thread_id": "10"}}
)

pprint(response)

{'favourite_colour': 'green',
 'messages': [HumanMessage(content='Hello, how are you?', additional_kwargs={}, response_metadata={}, id='cdc1e628-aaa4-4864-832f-f539a68ff141'),
              AIMessage(content='I don\'t have a specific function to call in this case, as there is no function provided that matches the context of the conversation. The function provided, "update_favourite_colour", is not relevant to the greeting.', additional_kwargs={}, response_metadata={'role': 'assistant', 'content': 'I don\'t have a specific function to call in this case, as there is no function provided that matches the context of the conversation. The function provided, "update_favourite_colour", is not relevant to the greeting.', 'refusal': None, 'annotations': None, 'audio': None, 'function_call': None, 'tool_calls': [], 'reasoning': None, 'reasoning_content': None, 'token_usage': {'prompt_tokens': 256, 'total_tokens': 301, 'completion_tokens': 45, 'prompt_tokens_details': None}, 'finish_reason': 'sto

## Read state

In [10]:
@tool
def read_favourite_colour(runtime: ToolRuntime) -> str:
    """Read the favourite colour of the user from the state."""
    try:
        return runtime.state["favourite_colour"]
    except KeyError:
        return "No favourite colour found in state"

agent = create_agent(
    model=model,
    tools=[update_favourite_colour, read_favourite_colour],
    checkpointer=InMemorySaver(),
    state_schema=CustomState
)

In [11]:
response = agent.invoke(
    { "messages": [HumanMessage(content="My favourite colour is green")]},
    {"configurable": {"thread_id": "1"}}
)

pprint(response)

{'favourite_colour': 'green',
 'messages': [HumanMessage(content='My favourite colour is green', additional_kwargs={}, response_metadata={}, id='08b9b137-0ee2-43e2-a4aa-05b9133fd3d9'),
              AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'chatcmpl-tool-bd045cdfae807680', 'type': 'function', 'function': {'name': 'update_favourite_colour', 'arguments': '{"favourite_colour": "green"}'}}]}, response_metadata={'role': 'assistant', 'content': None, 'refusal': None, 'annotations': None, 'audio': None, 'function_call': None, 'tool_calls': [{'id': 'chatcmpl-tool-bd045cdfae807680', 'type': 'function', 'function': {'name': 'update_favourite_colour', 'arguments': '{"favourite_colour": "green"}'}}], 'reasoning': None, 'reasoning_content': None, 'token_usage': {'prompt_tokens': 316, 'total_tokens': 343, 'completion_tokens': 27, 'prompt_tokens_details': None}, 'finish_reason': 'tool_calls', 'model_name': 'meta/llama-3.3-70b-instruct'}, id='lc_run--019c8d7d-3e42-7463-91ba-a0249

In [12]:
response = agent.invoke(
    { "messages": [HumanMessage(content="What's my favourite colour?")]},
    {"configurable": {"thread_id": "1"}}
)

pprint(response)

{'favourite_colour': 'green',
 'messages': [HumanMessage(content='My favourite colour is green', additional_kwargs={}, response_metadata={}, id='08b9b137-0ee2-43e2-a4aa-05b9133fd3d9'),
              AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'chatcmpl-tool-bd045cdfae807680', 'type': 'function', 'function': {'name': 'update_favourite_colour', 'arguments': '{"favourite_colour": "green"}'}}]}, response_metadata={'role': 'assistant', 'content': None, 'refusal': None, 'annotations': None, 'audio': None, 'function_call': None, 'tool_calls': [{'id': 'chatcmpl-tool-bd045cdfae807680', 'type': 'function', 'function': {'name': 'update_favourite_colour', 'arguments': '{"favourite_colour": "green"}'}}], 'reasoning': None, 'reasoning_content': None, 'token_usage': {'prompt_tokens': 316, 'total_tokens': 343, 'completion_tokens': 27, 'prompt_tokens_details': None}, 'finish_reason': 'tool_calls', 'model_name': 'meta/llama-3.3-70b-instruct'}, id='lc_run--019c8d7d-3e42-7463-91ba-a0249